# OR case 1

In [ ]:
import pandas as pd
import openpyxl
from gurobipy import *
import matplotlib.pyplot as plt
import numpy as np
import csv

In [ ]:
# which test set
i = 7
root = '../tests/test'
subfileName = ".csv"

tardyamount2_2_2 = 0
path = root+ str(i) + subfileName
print ('path:', path)

In [ ]:
import plotly.express as px
import plotly.io as pio
pio.renderers
pio.renderers.default = "notebook_connected"

# display settings
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# 第二、三題的甘特圖
def gantt_plot_2_3(x_opt, P, y_opt, problem_no, open_hour=0, open_min=0):
    import datetime

    I = len(x_opt)
    S = len(x_opt[0])
    M = len(y_opt[0][0])
    schedule_df = pd.DataFrame()
    open_time = datetime.datetime(2000, 1, 1, hour=open_hour, minute=open_min)
    schedule_df['START_TIME'] = [(open_time + datetime.timedelta(hours=x_opt[i][s]) - datetime.timedelta(hours=P[i][s])) for i in range(I) for s in range(S)]
    schedule_df['END_TIME'] = [(open_time + datetime.timedelta(hours=x_opt[i][s])) for i in range(I) for s in range(S)]

    machine_ID = []
    for i in range(I):
        for s in range(S):
            for m in range(M):
                if y_opt[i][s][m] == 1:
                    machine_ID.append(str(m + 1))
    schedule_df['Machine_ID'] = machine_ID

    schedule_df['Job_ID'] = [str(i + 1) for i in range(I) for s in range(S)]
    color_list = ["#C0392B", "#EC7063", "#1F618D", "#34495E", "#3498DB", "#1E8449", "#28B463", "#F1C40F", "#F39C12", "#A6ACAF", "#7D3C98", "#9B59B6", "#2C3E50", "#EF553B", "#636EFA", "#BDC3C7", "#F39C12"]
    fig = px.timeline(schedule_df, x_start="START_TIME", x_end="END_TIME", y="Machine_ID", color="Job_ID",
                      color_discrete_sequence=color_list[0:I],
                      category_orders={"Job_ID": [str(i+1) for i in range(I)], "Machine_ID": [str(m+1) for m in range(M)]},
                      title="Instance " + str(1) + " Problem " + str(problem_no) + " Gantt Chart")
    fig.update_xaxes(tickformat="%H:%M")
    fig.update_layout(legend_title="Lot ID")
    fig.show()
    fig.write_image("Instance " + str(1) + " Problem " + str(problem_no) + ".pdf")

In [ ]:
# which test set
i = 0

### 3.

### Constant Data

In [ ]:
from huristic1_1_2 import huristic1_1_2

makespan0, tardyamount0 = huristic1_1_2(path)

print('tardyamount', tardyamount0)
print('makespan', makespan0)

In [ ]:
from huristicPaperStage1insertFix import huristicPaperStage1InsertFix

makespan1, tardyamount1 = huristicPaperStage1InsertFix(path)

print('tardyamount', tardyamount1)
print('makespan', makespan1)

In [ ]:
# df['Stage-1 Machines']
# df['Stage-2 Machines']

In [ ]:
max_amount = 0
for index, row in df.iterrows():
    x1 = row['Stage-1 Machines'].split(',')
    for element in x1:
        num = int(element)
        if(num > max_amount):
            max_amount = num
            
    temp = row['Stage-2 Machines']
    if(pd.isna(temp) == False):
        x2 = temp.split(',')
        for element in x2:
            num = int(element)
            if(num > max_amount):
                max_amount = num

In [ ]:
max_amount

In [ ]:
# instance i 中 job 的總數量
I = len(df) # order num
S = 2 # stage num


orders = range(0, I)
machines = range(0, max_amount)
stages = range(0, S)

In [ ]:
P = []
for index, row in df.iterrows():
    P.append([])
    P[index].append(row['Stage-1 Processing Time'])
    P[index].append(row['Stage-2 Processing Time'])

In [ ]:
P

In [ ]:
# C_ijm -> init 3 dim 都是 false 的東西
C =  []
for i in range(I):
    C.append([])
    C[i].append([0 for i in range(max_amount)])
    C[i].append([0 for i in range(max_amount)])

In [ ]:
for index, row in df.iterrows():
    x1 = row['Stage-1 Machines'].split(',')
    for element in x1:
        num = int(element)
        C[index][0][num-1] = 1

        temp = row['Stage-2 Machines']
        if(pd.isna(temp) == False):
            x2 = temp.split(',')
            for element in x2:
                num = int(element)
                C[index][1][num-1] = 1

In [ ]:
C

In [ ]:
D = []
for index, row in df.iterrows():
    D.append(row['Due Time'])
D

### Decision variable – Delay

In [ ]:
Q3_delay = Model('Q3-delay')
Q3_delay.setParam('TimeLimit',100)

# x_is is the finish time of order i at stage s
x = {}
for i in orders:
    x[i] = {}
    for s in stages:
        x[i][s] = Q3_delay.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "x_" + str(i) + "," + str(s))

# y_ism = 1 if order i process on machine m  at stage s otherwise 0
y = {}
for i in orders:
    y[i] = {}
    for s in stages:
        y[i][s] = {}
        for m in machines:
            y[i][s][m] = Q3_delay.addVar(lb = 0, vtype = GRB.BINARY, name = "y_" + str(i) + "," + str(s) + "," + str(m))
        
# z_isjt = 1 if order i's stage s process before order j's stage t and they both process on the same machine otherwise 0
z = {}
for i in orders:
    z[i] = {}
    for s in stages:
        z[i][s] = {}
        for j in orders:
            z[i][s][j] = {}
            for t in stages:
                z[i][s][j][t] = Q3_delay.addVar(lb = 0, vtype = GRB.BINARY, name = "z_" + str(i) + "," + str(s) + "," + str(j) + "," + str(t))
        
# l_i = 1 if order i is late otherwise 0
l = {}
for i in orders:
    l[i] = Q3_delay.addVar(lb = 0, vtype = GRB.BINARY, name = "l_" + str(i))

Q3_delay.update()

### Setting the objective function

In [ ]:
Q3_delay.setObjective(quicksum(l[i] for i in orders), GRB.MINIMIZE)

### Add constraints

In [ ]:
Q3_delay.addConstrs((x[j][t] + P[i][s] - x[i][s] <= (quicksum(P[i][s] for i in orders for s in stages)+P[i][s]) * (z[i][s][j][t] + 2-y[i][s][m]-y[j][t][m]) for i in orders for j in range(i+1, I) for m in machines for s in stages for t in stages), 'constraints for x_is, z_isjt')
Q3_delay.addConstrs((x[i][s] + P[j][t] - x[j][t] <= (quicksum(P[i][s] for i in orders for s in stages)+P[j][t]) * (1-z[i][s][j][t] + 2-y[i][s][m]-y[j][t][m]) for i in orders for j in range(i+1, I) for m in machines for s in stages for t in stages), 'constraints for x_is, z_isjt')
Q3_delay.addConstrs((x[i][1] <= D[i] + quicksum(P[i][s] for s in stages for i in orders) * l[i] for i in orders), 'constraints for l_i')
Q3_delay.addConstrs((quicksum(y[i][s][m] for m in machines) == 1 for i in orders for s in stages), 'allocate orders to machines')
Q3_delay.addConstrs((x[i][s] >= P[i][s] for i in orders for s in stages), 'minimum x_i')
Q3_delay.addConstrs((x[i][1] >= x[i][0] + P[i][1] for i in orders), 'stage 2 should start after stage 1 is finished')

# C[i][s][m]
Q3_delay.addConstrs((y[i][s][m] <= (1 if C[i][s][m] == 0 else 1) for i in orders for s in stages for m in machines), 'constraints for cook machines')

Q3_delay.update()

In [ ]:
Q3_delay.optimize()

In [ ]:
a = Q3_delay.objVal
print("objective value =", a)

### Decision variable – makespan

In [ ]:
Q3_makespan = Model('Q3-makespan')
Q3_makespan.setParam('TimeLimit',100)

# x_is is the finish time of order i at stage s
x = {}
for i in orders:
    x[i] = {}
    for s in stages:
        x[i][s] = Q3_makespan.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "x_" + str(i) + "," + str(s))

# y_ism = 1 if order i process on machine m  at stage s otherwise 0
y = {}
for i in orders:
    y[i] = {}
    for s in stages:
        y[i][s] = {}
        for m in machines:
            y[i][s][m] = Q3_makespan.addVar(lb = 0, vtype = GRB.BINARY, name = "y_" + str(i) + "," + str(s) + "," + str(m))
        
# z_isjt = 1 if order i's stage s process before order j's stage t and they both process on the same machine otherwise 0
z = {}
for i in orders:
    z[i] = {}
    for s in stages:
        z[i][s] = {}
        for j in orders:
            z[i][s][j] = {}
            for t in stages:
                z[i][s][j][t] = Q3_makespan.addVar(lb = 0, vtype = GRB.BINARY, name = "z_" + str(i) + "," + str(s) + "," + str(j) + "," + str(t))
        
# l_i = 1 if order i is late otherwise 0
l = {}
for i in orders:
    l[i] = Q3_makespan.addVar(lb = 0, vtype = GRB.BINARY, name = "l_" + str(i))

# w is the makespan of the solution
w = Q3_makespan.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "w")

Q3_makespan.update()

### Setting the objective function

In [ ]:
Q3_makespan.setObjective(w, GRB.MINIMIZE)

### Add constraints

In [ ]:
Q3_makespan.addConstrs((x[j][t] + P[i][s] - x[i][s] <= (quicksum(P[i][s] for i in orders for s in stages)+P[i][s]) * (z[i][s][j][t] + 2-y[i][s][m]-y[j][t][m]) for i in orders for j in range(i+1, I) for m in machines for s in stages for t in stages), 'constraints for x_is, z_isjt')
Q3_makespan.addConstrs((x[i][s] + P[j][t] - x[j][t] <= (quicksum(P[i][s] for i in orders for s in stages)+P[j][t]) * (1-z[i][s][j][t] + 2-y[i][s][m]-y[j][t][m]) for i in orders for j in range(i+1, I) for m in machines for s in stages for t in stages), 'constraints for x_is, z_isjt')
Q3_makespan.addConstrs((x[i][1] <= D[i] + quicksum(P[i][s] for s in stages for i in orders) * l[i] for i in orders), 'constraints for l_i')
Q3_makespan.addConstrs((quicksum(y[i][s][m] for m in machines) == 1 for i in orders for s in stages), 'allocate orders to machines')
Q3_makespan.addConstrs((x[i][s] >= P[i][s] for i in orders for s in stages), 'minimum x_i')
Q3_makespan.addConstrs((x[i][1] >= x[i][0] + P[i][1] for i in orders), 'stage 2 should start after stage 1 is finished')
# C_ism
Q3_makespan.addConstrs((y[i][s][m] <= (1 if C[i][s][m] == 0 else 1) for i in orders for s in stages for m in machines), 'constraints for cook machines')
Q3_makespan.addConstrs((w >= x[i][1] for i in orders), 'makespan')
Q3_makespan.addConstr((quicksum(l[i] for i in orders) == a), 'delay num')
# Q3_makespan.addConstr((w == 6.1), 'delay num')

Q3_makespan.update()

In [ ]:
Q3_makespan.optimize()

In [ ]:
print("objective value =", Q3_makespan.objVal)

## Results

In [ ]:
x_opt = {}
for i in orders:
    x_opt[i] = {}
    for s in stages:
        x_opt[i][s] = {}

y_opt = {}
for i in orders:
    y_opt[i] = {}
    for s in stages:
        y_opt[i][s] = {}
        for m in machines:
            y_opt[i][s][m] = {}

for var in Q3_makespan.getVars():
    index = [i for i in var.varName[2:].split(',')]
    if var.varName[0] == 'x':
        x_opt[int(index[0])][int(index[1])] = var.x
    elif var.varName[0] == 'y':
        y_opt[int(index[0])][int(index[1])][int(index[2])] = var.x

## Plot Gantt Charts

In [ ]:
gantt_plot_2_3(x_opt, P, y_opt, 3)

In [ ]:
#optimality_gap for huristic1_1_2
AbsoluteErrorTardy = Q3_delay.objVal - tardyamount0
if(Q3_delay.objVal == 0):
    PercentageErrorTardy = 0
else:
    PercentageErrorTardy = (Q3_delay.objVal - tardyamount0)/float(Q3_delay.objVal)
    
AbsoluteErrorMakespan = Q3_makespan.objVal - makespan0
PercentageErrorMakespan = (Q3_makespan.objVal - makespan0)/float(Q3_makespan.objVal)
print('Absolute_Error_Tardy =', AbsoluteErrorTardy)
print('Percentage_Error_Tardy =', PercentageErrorTardy)
print('Absolute_Error_Makespan =', AbsoluteErrorMakespan)
print('Percentage_Error_Makespan =', PercentageErrorMakespan)
print(AbsoluteErrorTardy)
print(PercentageErrorTardy)
print(AbsoluteErrorMakespan)
print(PercentageErrorMakespan)


In [ ]:
#optimality_gap for huristic_paper
AbsoluteErrorTardy1 = Q3_delay.objVal - tardyamount1
if(Q3_delay.objVal == 0):
    PercentageErrorTardy1 = 0
else:
    PercentageErrorTardy1 = (Q3_delay.objVal - tardyamount1)/float(Q3_delay.objVal)
    
AbsoluteErrorMakespan1 = Q3_makespan.objVal - makespan1
PercentageErrorMakespan1 = (Q3_makespan.objVal - makespan1)/float(Q3_makespan.objVal)
print('Absolute_Error_Tardy =', AbsoluteErrorTardy1)
print('Percentage_Error_Tardy =', PercentageErrorTardy1)
print('Absolute_Error_Makespan =', AbsoluteErrorMakespan1)
print('Percentage_Error_Makespan =', PercentageErrorMakespan1)
print(AbsoluteErrorTardy1)
print(PercentageErrorTardy1)
print(AbsoluteErrorMakespan1)
print(PercentageErrorMakespan1)